In [12]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import modin.pandas as pd
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
import numpy as np
from matplotlib import pyplot as ptl
import seaborn as sns
import sys, os
sys.path.append("../module/")
from lipinski import * 
import ray


In [13]:
pd.set_option('display.max_columns', None)
ray.init()

2022-01-31 21:17:44,658	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.22.1.50',
 'raylet_ip_address': '10.22.1.50',
 'redis_address': '10.22.1.50:6379',
 'object_store_address': '/tmp/ray/session_2022-01-31_21-17-40_918130_348993/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-31_21-17-40_918130_348993/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-01-31_21-17-40_918130_348993',
 'metrics_export_port': 51297,
 'node_id': '20328e608ae07f5c5d552518ccd58655646481c405ed6202812688ae'}

In [8]:
con = sqlite3.connect("../data/chembl_29.db")

In [18]:
molregno_chembl_id = pd.read_sql_query("SELECT compound_properties.*, molecule_dictionary.chembl_id, compound_structures.canonical_smiles FROM compound_properties INNER JOIN compound_structures ON compound_structures.molregno=compound_properties.molregno INNER JOIN molecule_dictionary ON molecule_dictionary.molregno=compound_properties.molregno;", con)

To request implementation, send an email to feature_requests@modin.org.


In [19]:
molregno_chembl_id.head()

OptionError: 'Pattern matched multiple keys'

   molregno  mw_freebase  alogp  hba  hbd_lipinski  \
0         1       341.75   2.11  5.0           1.0   
1         2       332.32   1.33  6.0           1.0   
2         3       357.80   2.27  5.0           2.0   
3         4       307.31   1.46  5.0           1.0   
4         5       341.75   2.11  5.0           1.0   

   num_lipinski_ro5_violations     chembl_id  \
0                          0.0    CHEMBL6329   
1                          0.0    CHEMBL6328   
2                          0.0  CHEMBL265667   
3                          0.0    CHEMBL6362   
4                          0.0  CHEMBL267864   

[5 rows x 25 columns]

In [20]:
molregno_chembl_id = molregno_chembl_id.drop(['ro3_pass', 'hba', 'hbd', 'num_ro5_violations', 'full_molformula'], axis=1)

In [23]:
molregno_chembl_id.head()

OptionError: 'Pattern matched multiple keys'

   molregno  mw_freebase  alogp  num_lipinski_ro5_violations     chembl_id  \
0         1       341.75   2.11                          0.0    CHEMBL6329   
1         2       332.32   1.33                          0.0    CHEMBL6328   
2         3       357.80   2.27                          0.0  CHEMBL265667   
3         4       307.31   1.46                          0.0    CHEMBL6362   
4         5       341.75   2.11                          0.0  CHEMBL267864   

[5 rows x 20 columns]

In [6]:
molregno_chembl_id = pd.merge(molregno_chembl_id.chembl_id, molregno_chembl_id)

In [7]:
molregno_chembl_id = molregno_chembl_id.drop(['molregno'], axis=1)

In [8]:
molregno_chembl_id

OptionError: 'Pattern matched multiple keys'

             chembl_id  mw_freebase  alogp  hbd_lipinski  \
0           CHEMBL6329       341.75   2.11           1.0   
1           CHEMBL6328       332.32   1.33           1.0   
2         CHEMBL265667       357.80   2.27           2.0   
3           CHEMBL6362       307.31   1.46           1.0   
4         CHEMBL267864       341.75   2.11           1.0   
...                ...          ...    ...           ...   
2084719  CHEMBL4651250       556.38    NaN           NaN   
2084720  CHEMBL4651251       378.30   3.13           3.0   
2084721  CHEMBL4651252       356.42   2.29           4.0   
2084722  CHEMBL4651253      1023.94    NaN           NaN   
2084723  CHEMBL4651254       482.53   4.63           4.0   

         num_lipinski_ro5_violations  \
0                                0.0   
1                                0.0   
2                                0.0   
3                                0.0   
4                                0.0   
...                              ...   

In [16]:
index_duplicates = molregno_chembl_id[molregno_chembl_id.canonical_smiles.duplicated()].sort_values(by=['canonical_smiles']).index

In [17]:
molregno_chembl_id = molregno_chembl_id.drop(labels=index_duplicates, axis=0)
molregno_chembl_id

OptionError: 'Pattern matched multiple keys'

             chembl_id  mw_freebase  alogp  hbd_lipinski  \
0           CHEMBL6329       341.75   2.11           1.0   
1           CHEMBL6328       332.32   1.33           1.0   
2         CHEMBL265667       357.80   2.27           2.0   
3           CHEMBL6362       307.31   1.46           1.0   
4         CHEMBL267864       341.75   2.11           1.0   
...                ...          ...    ...           ...   
2084719  CHEMBL4651250       556.38    NaN           NaN   
2084720  CHEMBL4651251       378.30   3.13           3.0   
2084721  CHEMBL4651252       356.42   2.29           4.0   
2084722  CHEMBL4651253      1023.94    NaN           NaN   
2084723  CHEMBL4651254       482.53   4.63           4.0   

         num_lipinski_ro5_violations  \
0                                0.0   
1                                0.0   
2                                0.0   
3                                0.0   
4                                0.0   
...                              ...   

In [11]:
ids_com_nan = molregno_chembl_id.loc[molregno_chembl_id.isnull().any(axis=1)][['chembl_id', 'canonical_smiles']]

In [12]:
ids_com_nan

,chembl_id,canonical_smiles
0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...
2084716,CHEMBL4651247,CN1CCN(c2cc3c(cc2F)c(=O)c(-c2nnc4sc(-c5cn(C6CC...
2084717,CHEMBL4651248,CN(C)C(=O)Oc1cccc(C(O)CNc2ccc(F)cc2)c1
2084719,CHEMBL4651250,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,CHEMBL4651251,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2


In [13]:
atualiza_data_frame_com_lipinski(molregno_chembl_id, ids_com_nan)

KeyboardInterrupt: 

In [13]:
#or chemb, smiles in zip(ids_com_nan.chembl_id, ids_com_nan.canonical_smiles):
 #   propriedades = verifica_lipinsky(smiles)
 ##   for key in propriedades.keys():
  #      molregno_chembl_id.loc[molregno_chembl_id.chembl_id == chemb, key] = propriedades[key]

KeyError: 'hbd'

In [ ]:
molregno_chembl_id

In [16]:
molregno_chembl_id.loc[(molregno_chembl_id.isnull().any(axis=1))]

AttributeError: 'int' object has no attribute 'loc'

In [12]:
molregno_chembl_id

,chembl_id,mw_freebase,alogp,psa,rtb,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,canonical_smiles
0,CHEMBL6329,341.75,2.11,84.82,3.0,6.48,NaN,3.63,2.69,ACID,341.75,3.0,24.0,0.74,341.0567,6.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,CHEMBL6328,332.32,1.33,108.61,3.0,6.33,NaN,2.88,1.82,ACID,332.32,3.0,25.0,0.73,332.0909,7.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,CHEMBL265667,357.80,2.27,87.98,3.0,6.33,NaN,3.70,2.64,ACID,357.80,3.0,25.0,0.75,357.0880,6.0,2.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,CHEMBL6362,307.31,1.46,84.82,3.0,6.33,NaN,3.02,1.97,ACID,307.31,3.0,23.0,0.74,307.0957,6.0,1.0,0.0,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,CHEMBL267864,341.75,2.11,84.82,3.0,6.33,NaN,3.63,2.57,ACID,341.75,3.0,24.0,0.74,341.0567,6.0,1.0,0.0,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084719,CHEMBL4651250,556.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,556.38,NaN,NaN,NaN,556.2055,NaN,NaN,NaN,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,CHEMBL4651251,378.30,3.13,58.36,5.0,NaN,9.33,2.96,0.43,BASE,378.30,2.0,25.0,0.84,377.1062,4.0,3.0,0.0,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2
2084721,CHEMBL4651252,356.42,2.29,90.82,7.0,9.18,9.79,1.64,0.31,BASE,356.42,2.0,26.0,0.57,356.1736,6.0,4.0,0.0,Cc1ccc(CCCNCC(O)c2cc(O)cc3c2OCC(=O)N3)cc1
2084722,CHEMBL4651253,1023.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1023.94,NaN,NaN,NaN,1023.2919,NaN,NaN,NaN,CCCCCCCCCCCCCCCCOC(=O)Oc1ccc(COP(=O)(OCc2ccc(O...


In [114]:
molregno_chembl_id.query("molregno in @ids_com_nan")

,molregno,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,chembl_id,canonical_smiles
0,1,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.48,NaN,3.63,2.69,ACID,341.75,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,CHEMBL6329,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccccc1Cl
1,2,332.32,1.33,6.0,1.0,108.61,3.0,N,0.0,6.33,NaN,2.88,1.82,ACID,332.32,3.0,25.0,0.73,332.0909,C18H12N4O3,7.0,1.0,0.0,CHEMBL6328,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(C#N)cc1
2,3,357.80,2.27,5.0,2.0,87.98,3.0,N,0.0,6.33,NaN,3.70,2.64,ACID,357.80,3.0,25.0,0.75,357.0880,C18H16ClN3O3,6.0,2.0,0.0,CHEMBL265667,Cc1cc(-n2ncc(=O)[nH]c2=O)cc(C)c1C(O)c1ccc(Cl)cc1
3,4,307.31,1.46,5.0,1.0,84.82,3.0,N,0.0,6.33,NaN,3.02,1.97,ACID,307.31,3.0,23.0,0.74,307.0957,C17H13N3O3,6.0,1.0,0.0,CHEMBL6362,Cc1ccc(C(=O)c2ccc(-n3ncc(=O)[nH]c3=O)cc2)cc1
4,5,341.75,2.11,5.0,1.0,84.82,3.0,N,0.0,6.33,NaN,3.63,2.57,ACID,341.75,3.0,24.0,0.74,341.0567,C17H12ClN3O3,6.0,1.0,0.0,CHEMBL267864,Cc1cc(-n2ncc(=O)[nH]c2=O)ccc1C(=O)c1ccc(Cl)cc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084716,2487507,725.83,4.00,14.0,0.0,112.93,6.0,N,2.0,NaN,6.55,4.64,4.58,NEUTRAL,725.83,6.0,52.0,0.25,725.2820,C36H37F2N11O2S,13.0,0.0,2.0,CHEMBL4651247,CN1CCN(c2cc3c(cc2F)c(=O)c(-c2nnc4sc(-c5cn(C6CC...
2084717,2487508,318.35,3.03,4.0,2.0,61.80,5.0,N,0.0,NaN,4.00,2.40,2.40,NEUTRAL,318.35,2.0,23.0,0.89,318.1380,C17H19FN2O3,5.0,2.0,0.0,CHEMBL4651248,CN(C)C(=O)Oc1cccc(C(O)CNc2ccc(F)cc2)c1
2084719,2487510,556.38,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,556.38,NaN,NaN,NaN,556.2055,C31H32BNO6P+,NaN,NaN,NaN,CHEMBL4651250,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...
2084720,2487511,378.30,3.13,3.0,2.0,58.36,5.0,N,0.0,NaN,9.33,2.96,0.43,BASE,378.30,2.0,25.0,0.84,377.1062,C19H21Cl2N3O,4.0,3.0,0.0,CHEMBL4651251,CNCc1ccc2c(c1)CN(C(=O)[C@H](N)Cc1ccc(Cl)cc1Cl)C2


In [63]:
molregno_chembl_id.query("num_ro5_violations != num_lipinski_ro5_violations")

,molregno,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,chembl_id,canonical_smiles
36,58,576.65,2.96,10.0,2.0,149.21,11.0,N,1.0,NaN,7.24,3.70,3.48,NEUTRAL,576.65,3.0,42.0,0.20,576.2696,C30H36N6O6,12.0,3.0,2.0,CHEMBL266223,COC(=O)/C=C/C(=O)Nc1ccc(CCCCC(=O)N2CCN(c3nc(N)...
47,69,546.58,2.26,10.0,2.0,149.21,8.0,N,1.0,NaN,7.24,2.89,2.67,NEUTRAL,546.58,3.0,40.0,0.32,546.2227,C28H30N6O6,12.0,3.0,2.0,CHEMBL6219,COC(=O)/C=C/C(=O)Nc1ccc(/C=C/C(=O)N2CCN(c3nc(N...
80,102,465.54,3.91,8.0,2.0,137.86,5.0,N,0.0,-12.17,14.51,5.04,5.17,ZWITTERION,465.54,3.0,33.0,0.15,465.1695,C21H23N9O2S,11.0,2.0,1.0,CHEMBL6313,COc1cc2nc(N3CCN(/C(S)=N/c4ccc(N=[N+]=[N-])cc4)...
94,117,617.70,3.62,10.0,1.0,134.33,12.0,N,1.0,13.88,NaN,2.53,2.53,NEUTRAL,617.70,2.0,44.0,0.38,617.2948,C31H43N3O10,13.0,1.0,2.0,CHEMBL266886,CCC(C)(C)NC(=O)OCC1CN(C(=O)c2cc(OC)c(OC)c(OC)c...
95,118,631.72,3.87,10.0,1.0,134.33,13.0,N,1.0,NaN,NaN,2.98,2.98,NEUTRAL,631.72,2.0,45.0,0.35,631.3105,C32H45N3O10,13.0,1.0,2.0,CHEMBL269132,COc1cc(C(=O)N2CCN(C(=O)c3cc(OC)c(OC)c(OC)c3)C(...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084668,2487459,713.59,5.81,9.0,1.0,117.68,8.0,N,2.0,8.23,NaN,1.48,1.43,NEUTRAL,713.59,6.0,47.0,0.20,712.0860,C34H27BrN5O6S+,11.0,1.0,3.0,CHEMBL4651199,COc1ccc2cc3[n+](cc2c1OCc1cn(-c2ccc(S(=O)(=O)Nc...
2084678,2487469,610.60,2.00,10.0,2.0,185.69,10.0,N,1.0,3.45,NaN,0.74,-2.65,ACID,610.60,2.0,43.0,0.17,610.1370,C28H26N4O10S,14.0,2.0,2.0,CHEMBL4651209,COc1ccc(N2C(=O)C(CC(=O)N[C@@H]3C(=O)N4C(C(=O)O...
2084691,2487482,519.46,-1.04,10.0,6.0,207.71,8.0,N,2.0,2.56,1.83,-4.34,-5.16,ACID,519.46,4.0,36.0,0.12,519.1631,C21H26N7O7P,14.0,7.0,3.0,CHEMBL4651222,C[n+]1cn([C@@H]2O[C@H](COP(=O)(O)NCCc3c[nH]c4c...
2084719,2487510,556.38,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,556.38,NaN,NaN,NaN,556.2055,C31H32BNO6P+,NaN,NaN,NaN,CHEMBL4651250,COc1ccc([P+](CCC(=O)Nc2ccc3c(c2)B(O)OC3)(c2ccc...


In [ ]:
cruzain_id = pd.read_csv('data/cruzain_id_chemb.csv')

In [ ]:
cruzain_id

In [ ]:
cruzain_id = cruzain_id["Molecule ChEMBL ID"]
cruzain_id

In [ ]:
molregno_chembl_id = molregno_chembl_id.query("chembl_id in @cruzain_id")

In [ ]:
molregno_chembl_id.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
molregno_chembl_id.num_lipinski_ro5_violations.unique()

In [ ]:
molregno_chembl_id.num_ro5_violations.unique()

In [ ]:
molregno_chembl_id[molregno_chembl_id.num_lipinski_ro5_violations == molregno_chembl_id.num_ro5_violations]

In [ ]:
molregno_chembl_id['ro3_pass'] = molregno_chembl_id['ro3_pass'].replace({"Y": 'True', "N": 'False'})

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0")

In [ ]:
molregno_chembl_id.query("num_lipinski_ro5_violations == 0 & ro3_pass == 'True'")

In [ ]:
prep_clust = pd.get_dummies(molregno_chembl_id.query("num_lipinski_ro5_violations == 0"), columns=["molecular_species"])
tab_usada = prep_clust
prep_clust = prep_clust.drop(['chembl_id','molregno', "full_molformula", "canonical_smiles"], axis=1)


In [ ]:
prep_clust.head()

In [ ]:
prep_clust = prep_clust.replace({np.nan: "7"})
prep_clust.head()

In [ ]:
x = preprocessing.normalize(prep_clust, norm='l1')
cluster = KMeans(n_clusters=3, random_state=5)
cluster.fit(x).labels_

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x=prep_clust[['mw_freebase']], y=prep_clust.psa, c=cluster.fit(x).labels_, s=50)
plt.xlabel('mw_freebase', fontsize=18)
plt.ylabel('psa', fontsize=18)
plt.show()

In [ ]:
tab_usada['grupos_cluster_knn'] = cluster.fit(x).labels_
tab_usada.drop(["num_ro5_violations", "ro3_pass", "num_lipinski_ro5_violations"], axis=1).sort_values(by='grupos_cluster_knn')

In [ ]:
sns.pairplot(tab_usada[['mw_freebase', 'psa', 'grupos_cluster_knn']], hue='grupos_cluster_knn')
